In [1]:
# Use a different environment
# Instead of opencv-python, use opencv-python-headless (requiring a different environment)

# https://github.com/edavalosanaya/plot3d
import plot3d

# In a separate terminal, run to start the server:
# plot3d

# Imports
import cv2
import time
import pathlib
import os
from tqdm import tqdm
import numpy as np
import imutils
import trimesh
from scipy.spatial.transform import Rotation as R
import pandas as pd

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent.parent
DATA_DIR = GIT_ROOT / "data" / 'AIED2024'

# Append ZoeDepth to path
import sys
sys.path.append('ZoeDepth')

/tmp/ipykernel_515129/691582857.py:20: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Create a plot
plot = plot3d.Plot(port=9010)

In [3]:
def get_intrinsics(H,W):
    """
    Intrinsics for a pinhole camera model.
    Assume fov of 55 degrees and central principal point.
    """
    f = 0.5 * W / np.tan(0.5 * 55 * np.pi / 180.0)
    cx = 0.5 * W
    cy = 0.5 * H
    return np.array([[f, 0, cx],
                     [0, f, cy],
                     [0, 0, 1]])

def depth_to_points(depth, R=None, t=None):

    K = get_intrinsics(depth.shape[1], depth.shape[2])
    Kinv = np.linalg.inv(K)
    if R is None:
        R = np.eye(3)
    if t is None:
        t = np.zeros(3)

    # M converts from your coordinate to PyTorch3D's coordinate system
    M = np.eye(3)
    M[0, 0] = -1.0
    M[1, 1] = -1.0

    height, width = depth.shape[1:3]

    x = np.arange(width)
    y = np.arange(height)
    coord = np.stack(np.meshgrid(x, y), -1)
    coord = np.concatenate((coord, np.ones_like(coord)[:, :, [0]]), -1)  # z=1
    coord = coord.astype(np.float32)
    # coord = torch.as_tensor(coord, dtype=torch.float32, device=device)
    coord = coord[None]  # bs, h, w, 3

    D = depth[:, :, :, None, None]
    # print(D.shape, Kinv[None, None, None, ...].shape, coord[:, :, :, :, None].shape )
    pts3D_1 = D * Kinv[None, None, None, ...] @ coord[:, :, :, :, None]
    # pts3D_1 live in your coordinate system. Convert them to Py3D's
    pts3D_1 = M[None, None, None, ...] @ pts3D_1
    # from reference to targe tviewpoint
    pts3D_2 = R[None, None, None, ...] @ pts3D_1 + t[None, None, None, :, None]
    # pts3D_2 = pts3D_1
    # depth_2 = pts3D_2[:, :, :, 2, :]  # b,1,h,w
    return pts3D_2[:, :, :, :3, 0][0]

def depth_edges_mask(depth):
    """Returns a mask of edges in the depth map.
    Args:
    depth: 2D numpy array of shape (H, W) with dtype float32.
    Returns:
    mask: 2D numpy array of shape (H, W) with dtype bool.
    """
    # Compute the x and y gradients of the depth map.
    depth_dx, depth_dy = np.gradient(depth)
    # Compute the gradient magnitude.
    depth_grad = np.sqrt(depth_dx ** 2 + depth_dy ** 2)
    # Compute the edge mask.
    mask = depth_grad > 0.05
    return mask

def create_triangles(h, w, mask=None):
    """Creates mesh triangle indices from a given pixel grid size.
        This function is not and need not be differentiable as triangle indices are
        fixed.
    Args:
    h: (int) denoting the height of the image.
    w: (int) denoting the width of the image.
    Returns:
    triangles: 2D numpy array of indices (int) with shape (2(W-1)(H-1) x 3)
    """
    x, y = np.meshgrid(range(w - 1), range(h - 1))
    tl = y * w + x
    tr = y * w + x + 1
    bl = (y + 1) * w + x
    br = (y + 1) * w + x + 1
    triangles = np.array([tl, bl, tr, br, tr, bl])
    triangles = np.transpose(triangles, (1, 2, 0)).reshape(
        ((w - 1) * (h - 1) * 2, 3))
    if mask is not None:
        mask = mask.reshape(-1)
        triangles = triangles[mask[triangles].all(1)]
    return triangles

def get_mesh(image, depth, keep_edges=False):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pts3d = depth_to_points(depth[None])
    pts3d = pts3d.reshape(-1, 3)

    # Create a trimesh mesh from the points
    # Each pixel is connected to its 4 neighbors
    # colors are the RGB values of the image

    verts = pts3d.reshape(-1, 3)
    image = np.array(image)
    if keep_edges:
        triangles = create_triangles(image.shape[0], image.shape[1])
    else:
        triangles = create_triangles(image.shape[0], image.shape[1], mask=~depth_edges_mask(depth))
    colors = image.reshape(-1, 3)
    mesh = trimesh.Trimesh(vertices=verts, faces=triangles, vertex_colors=colors)

    # Save as glb
    return mesh

def compute_3D_point(x, y, Z, H, W):
    """
    Compute the 3D point in the camera coordinate system from an image coordinate and depth.

    Parameters:
    - x, y: The image coordinates (pixels)
    - Z: The depth value (distance along the camera's viewing axis)
    - f_x, f_y: The camera's focal lengths along the X and Y axes (pixels)
    - c_x, c_y: The optical center of the camera (pixels)

    Returns:
    A tuple (X, Y, Z) representing the 3D point in the camera coordinate system.
    """
    # 
    fy = 0.5 * W / np.tan(0.5 * 55 * np.pi / 180.0)
    fx = 0.5 * W / np.tan(0.5 * 55 * np.pi / 180.0)
    cx = 0.5 * W
    cy = 0.5 * H

    # Normalize the 2D coordinates
    x_prime = (x - cx) / fx
    y_prime = (y - cy) / fy

    # Apply the depth to get the 3D point
    X = x_prime * Z
    Y = y_prime * Z

    return (X, Y, Z)


In [4]:
# Reset the 3D Plot
plot.reset()

# Add the monitor rectangle
SIZE = 125
r = R.from_rotvec(np.pi/2 * np.array([-0.2, 0.75, -0.1]))
t = np.array([4, 0, -10])*4
rt = np.eye(4)
rt[:3, :3] = r.as_matrix()
rt[:3, 3] = t

rect = trimesh.creation.box(extents=np.array([0.5, 0.2, 0.01])*SIZE)
rect.visual.face_colors = [1, 0, 0, 0.5]
rect.visual.vertex_colors = [1, 0, 0, 0.5]

rect.apply_transform(rt)
plot.add_mesh('monitor', rect)
# plot.update_mesh('monitor', rect)

sphere = trimesh.creation.uv_sphere(radius=0.5)
sphere.visual.face_colors = [0, 1, 0, 0.5]
sphere.visual.vertex_colors = [0, 1, 0, 0.5]

# Load the gaze vectors and the corresponding CSV with BBox info
faces_fp = DATA_DIR / 'Cam1_SampleData_Analysis' / 'C1Sample.csv'
assert faces_fp.exists()
faces_df = pd.read_csv(faces_fp)
gaze_vectors_dir = DATA_DIR / 'gaze_vectors'

# Load the RGB and depth videos
vid_file = DATA_DIR / "block-a-blue-day1-first-group-cam1.mp4"
assert vid_file.exists()
cap = cv2.VideoCapture(str(vid_file))
LENGTH = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

depth_file = DATA_DIR / 'depths' / 'd1g1' / "depth_test.mp4"
assert depth_file.exists()
depth_cap = cv2.VideoCapture(str(depth_file))

delete_after_step = []

for i in tqdm(range(LENGTH), total=LENGTH):

    # Load frame
    r_ret, rgb = cap.read()
    d_ret, depth = depth_cap.read()

    if not r_ret or not d_ret:
        break

    depth = cv2.cvtColor(depth, cv2.COLOR_BGR2GRAY)

    # Get the gaze vector
    faces = faces_df[faces_df['Frame'] == i]

    j = 0
    for (index, face) in faces.iterrows():
        # Get the gaze vector
        # gaze_vector_pitch_file = gaze_vectors_dir / f'frame{face.frame}_face{face.Face_Index}_pitch.npy'
        # gaze_vector_yaw_file = gaze_vectors_dir / f'frame{face.frame}_face{face.Face_Index}_yaw.npy'
        # pitch = np.load(gaze_vector_pitch_file)
        # yaw = np.load(gaze_vector_yaw_file)

        # Compute the centroid of the face
        centroid = (face.X + face.Width/2, face.Y + face.Height/2)
        centroid_depth = depth[int(centroid[1]), int(centroid[0])]*-1
        face_t = compute_3D_point(centroid[0], centroid[1], centroid_depth, depth.shape[0], depth.shape[1])

        # Make copy of spher and apply transform
        sphere_copy = sphere.copy()
        sphere_copy.apply_transform(trimesh.transformations.translation_matrix(face_t))

        # Draw spheres in the 3D plot
        if f"face-{j}" in plot.client.visuals:
            plot.update_mesh(f'face-{j}', sphere_copy)
        else:
            plot.add_mesh(f'face-{j}', sphere_copy)
        j += 1

        # Draw in the 2D image
        cv2.circle(rgb, (int(centroid[0]), int(centroid[1])), 5, (0, 255, 0), -1)
            
    # Resize
    sm_rgb = imutils.resize(rgb, width=500)
    sm_depth = imutils.resize(depth, width=500)

    mesh = get_mesh(sm_rgb, sm_depth, keep_edges=True)
    mesh.apply_transform(trimesh.transformations.rotation_matrix(np.pi, [1,0,0]))

    # Plot the frame
    plot.plot_image(sm_rgb)
    if i == 0:
        plot.add_mesh('mesh', mesh)
    else:
        plot.update_mesh('mesh', mesh)
        

  3%|▎         | 355/13464 [00:57<35:25,  6.17it/s]
